In [30]:
import math

def euclidean_distance(vec1, vec2):
    sum = 0
    for i in range(0, len(vec1)):
        sum += math.pow(vec1[i]-vec2[i],2)
    return math.sqrt(sum)


def manhattan_distance(vec1, vec2):
    sum = 0
    for i in range(0, len(vec1)):
        sum += math.fabs(vec1[i] - vec2[i])
    return sum

In [31]:


    
vec1 = [1, 2, 3]
vec2 = [4, 5, 6]
euclidean_distance(vec1, vec2)
manhattan_distance(vec1, vec2)

9.0

In [16]:
import numpy

numpy.linalg.norm(numpy.array(vec1) - numpy.array(vec2))

5.196152422706632

In [17]:
from scipy.spatial import distance

a = (1, 2, 3)
b = (4, 5, 6)
distance.euclidean(a, b)

5.196152422706632